In [1]:
# environment: thesisPlayground_pointClouds_env

I should need:
- image(s) (RGB, or the hdf5)
- depths
- intrinsics matrix
- camera poses (when having multiple images to put together)

In [2]:
import h5py
import numpy as np
import open3d as o3d
import pandas as pd
import glob
import os

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
def get_intrinsics_from_metadata(scene_row):
    """
    Derive the intrinsic matrix from metadata.

    Parameters:
        scene_row (pd.Series): Metadata row for the scene.

    Returns:
        np.ndarray: The intrinsic matrix K.
    """
    # Extract the necessary parameters from the metadata
    img_width = scene_row["settings_output_img_width"]
    img_height = scene_row["settings_output_img_height"]
    focal_length = scene_row["camera_physical_focal_length"]
    sensor_width = scene_row["camera_physical_film_width"]

    # Calculate fx and fy using the focal length and sensor width
    fx = focal_length / sensor_width * img_width
    fy = fx  # Assuming square pixels

    # Calculate the principal point (cx, cy) considering horizontal and vertical shifts
    cx = img_width / 2 + scene_row["camera_physical_horizontal_shift"]
    cy = img_height / 2 + scene_row["camera_physical_lens_shift"]

    # Construct the intrinsic matrix
    intrinsic_matrix = np.array([
        [fx, 0, cx],
        [0, fy, cy],
        [0, 0, 1]
    ])
    
    return intrinsic_matrix



def get_intrinsics_with_tilt_shift(path_metadata, scene_name):
    """
    Load camera metadata from a CSV file and compute the intrinsic matrix considering tilt-shift parameters.

    Parameters:
        path_metadata (str): Path to the metadata CSV file.
        scene_name (str): Name of the scene to analyze.

    Returns:
        np.ndarray: The updated intrinsic matrix K.
    """
    # Load the metadata CSV file
    df = pd.read_csv(path_metadata)
    
    # Filter the DataFrame to get the row corresponding to the specified scene
    scene_row = df[df['scene_name'] == scene_name]
    
    if scene_row.empty:
        raise ValueError(f"Scene '{scene_name}' not found in the metadata.")
    
    # Use the first matching row
    scene_row = scene_row.iloc[0]
    
    # Get the intrinsic matrix from metadata
    intrinsic_matrix = get_intrinsics_from_metadata(scene_row)
    
    return intrinsic_matrix


# TEST
path_metadata = '/local/home/gmarsich/Desktop/Thesis/0Code_playground/pointClouds/pointClouds_ChatGPT/metadata_camera_parameters.csv'
scene = 'ai_007_008'  # name of the scene, with format ai_VVV_NNN
intrinsics = get_intrinsics_with_tilt_shift(path_metadata, scene)
print("Intrinsic Matrix for the scene:")
print(intrinsics)

Intrinsic Matrix for the scene:
[[1.43810735e+03 0.00000000e+00 5.12000000e+02]
 [0.00000000e+00 1.43810735e+03 3.83920000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [4]:
# TODO: for multimple images

# def get_camera_pose(scene_row):
#     """
#     Extract the camera pose from the metadata row.

#     Parameters:
#         scene_row (pd.Series): Metadata row for the scene.

#     Returns:
#         np.ndarray: The camera extrinsic matrix (4x4).
#     """
#     # Extract rotation matrix
#     R = np.array([
#         [scene_row["M_cam_from_uv_00"], scene_row["M_cam_from_uv_01"], scene_row["M_cam_from_uv_02"]],
#         [scene_row["M_cam_from_uv_10"], scene_row["M_cam_from_uv_11"], scene_row["M_cam_from_uv_12"]],
#         [scene_row["M_cam_from_uv_20"], scene_row["M_cam_from_uv_21"], scene_row["M_cam_from_uv_22"]],
#     ])
    
#     # Extract translation vector
#     t = np.array([scene_row["M_proj_03"], scene_row["M_proj_13"], scene_row["M_proj_23"]])
    
#     # Construct the extrinsic matrix
#     extrinsic_matrix = np.eye(4)
#     extrinsic_matrix[:3, :3] = R
#     extrinsic_matrix[:3, 3] = t
    
#     return extrinsic_matrix


In [5]:
def load_hdf5_data(image_files, depth_files):
    """
    Load images and depth data from multiple HDF5 files.
    
    Parameters:
        image_files (list): List of paths to the HDF5 files containing images.
        depth_files (list): List of paths to the HDF5 files containing depth maps.
    
    Returns:
        images (list): List of images.
        depths (list): List of depth maps.
    """

    images = []
    depths = []

    for image_file, depth_file in zip(image_files, depth_files):
        with h5py.File(image_file, 'r') as f:
            images.append(np.array(f['dataset']))  # Adjust the key according to your HDF5 structure

        with h5py.File(depth_file, 'r') as f:
            depths.append(np.array(f['dataset']))  # Adjust the key according to your HDF5 structure

    return images, depths

In [6]:
#OLD
# def generate_point_cloud(images, depths, intrinsic_matrix):
#     """
#     Generate point clouds from images and depth maps.
    
#     Parameters:
#         images (list): List of images.
#         depths (list): List of depth maps.
#         intrinsic_matrix (np.ndarray): Intrinsic matrix of the camera.
    
#     Returns:
#         point_clouds (list): List of Open3D PointCloud objects.
#     """

#     point_clouds = []

#     for image, depth in zip(images, depths):
#         # Create an empty point cloud
#         points = []

#         h, w = depth.shape

#         u, v = np.meshgrid(np.arange(w), np.arange(h))
#         u = u.flatten()
#         v = v.flatten()

#         z = depth.flatten()

        
#         valid = z > 0
#         u = u[valid]
#         v = v[valid]
#         z = z[valid]

#         x = (u - intrinsic_matrix[0, 2]) / intrinsic_matrix[0, 0]
#         y = (v - intrinsic_matrix[1, 2]) / intrinsic_matrix[1, 1]

#         # C
#         x = x * z
#         y = y * z

#         points = np.vstack((x, y, z)).T
        
#         pcd = o3d.geometry.PointCloud()
#         pcd.points = o3d.utility.Vector3dVector(points)
#         point_clouds.append(pcd)
    
#     return point_clouds



# ORIGINAL:

# def generate_point_cloud(images, depths, intrinsic_matrix, extrinsic_matrices):
#     """
#     Generate point clouds from images and depth maps and apply the camera extrinsic transformations.

#     Parameters:
#         images (list): List of images.
#         depths (list): List of depth maps.
#         intrinsic_matrix (np.ndarray): Intrinsic matrix of the camera.
#         extrinsic_matrices (list): List of camera extrinsic matrices.

#     Returns:
#         point_clouds (list): List of Open3D PointCloud objects.
#     """
#     point_clouds = []

#     for image, depth, extrinsic_matrix in zip(images, depths, extrinsic_matrices):

#         # Get the image dimensions
#         h, w = depth.shape

#         # Create a mesh grid of pixel coordinates
#         u, v = np.meshgrid(np.arange(w), np.arange(h))
#         u = u.flatten()
#         v = v.flatten()

#         # Get the corresponding depth values
#         z = depth.flatten()

#         # Filter out points with zero depth
#         valid = z > 0
#         u = u[valid]
#         v = v[valid]
#         z = z[valid]

#         # Convert pixel coordinates to normalized image coordinates; create 3D points in the camera coordinate system
#         x = (u - intrinsic_matrix[0, 2]) / intrinsic_matrix[0, 0] * z
#         y = (v - intrinsic_matrix[1, 2]) / intrinsic_matrix[1, 1] * z

#         # Stack the coordinates into a single array
#         points = np.vstack((x, y, z)).T

#         # Apply extrinsic transformation
#         ones = np.ones((points.shape[0], 1))
#         points_homogeneous = np.hstack((points, ones))
#         points_transformed = (extrinsic_matrix @ points_homogeneous.T).T[:, :3]

#         # Create Open3D PointCloud object and add it to the list
#         pcd = o3d.geometry.PointCloud()
#         pcd.points = o3d.utility.Vector3dVector(points_transformed)
#         point_clouds.append(pcd)

#     return point_clouds



In [7]:
def generate_point_cloud(images, depths, intrinsic_matrix, extrinsic_matrices):
    """
    Generate point clouds from images and depth maps and apply the camera extrinsic transformations.

    Parameters:
        images (list): List of images.
        depths (list): List of depth maps.
        intrinsic_matrix (np.ndarray): Intrinsic matrix of the camera.
        extrinsic_matrices (list): List of camera extrinsic matrices.

    Returns:
        point_clouds (list): List of Open3D PointCloud objects.
    """
    point_clouds = []

    for image, depth in zip(images, depths):

        # Get the image dimensions
        h, w = depth.shape

        # Create a mesh grid of pixel coordinates
        u, v = np.meshgrid(np.arange(w), np.arange(h))
        u = u.flatten()
        v = v.flatten()

        # Get the corresponding depth values
        z = depth.flatten()

        # Filter out points with zero depth
        valid = z > 0
        u = u[valid]
        v = v[valid]
        z = z[valid]

        # Convert pixel coordinates to normalized image coordinates; create 3D points in the camera coordinate system
        x = (u - intrinsic_matrix[0, 2]) / intrinsic_matrix[0, 0] * z
        y = (v - intrinsic_matrix[1, 2]) / intrinsic_matrix[1, 1] * z

        # Stack the coordinates into a single array
        points = np.vstack((x, y, z)).T

        # # Apply extrinsic transformation
        # ones = np.ones((points.shape[0], 1))
        # points_homogeneous = np.hstack((points, ones))
        # points_transformed = (extrinsic_matrix @ points_homogeneous.T).T[:, :3]

        # Create Open3D PointCloud object and add it to the list
        pcd = o3d.geometry.PointCloud()
        #pcd.points = o3d.utility.Vector3dVector(points_transformed)
        pcd.points = o3d.utility.Vector3dVector(points)
        point_clouds.append(pcd)

    return point_clouds




# TEST
path_metadata = '/local/home/gmarsich/Desktop/Thesis/0Code_playground/pointClouds/pointClouds_ChatGPT/metadata_camera_parameters.csv'

base_path = '/local/home/gmarsich/data2TB/Hypersim/evermotion_dataset/scenes'
scene = 'ai_007_008'  # name of the scene, with format ai_VVV_NNN
cam_xx = 'cam_00'

# Get list of image and depth HDF5 files
image_files = sorted(glob.glob(os.path.join(base_path, scene, 'images', 'scene_' + cam_xx + '_final_hdf5', '*.color.hdf5')))
depth_files = sorted(glob.glob(os.path.join(base_path, scene, 'images', 'scene_' + cam_xx + '_geometry_hdf5', '*.depth_meters.hdf5')))

print(os.path.join(base_path, scene, 'images', 'scene_' + cam_xx + '_final_hdf5', '*.color.hdf5'))

print(image_files)

# Ensure the number of image and depth files match
if len(image_files) != len(depth_files):
    raise ValueError("The number of image files and depth files do not match.")

# Get the intrinsic matrix for the scene
intrinsics = get_intrinsics_with_tilt_shift(path_metadata, scene)
print("Intrinsic Matrix for the scene:")
print(intrinsics)

# Load images and depth data
images, depths = load_hdf5_data(image_files, depth_files)

# Generate point clouds for each view
point_clouds = generate_point_cloud(images, depths, intrinsics, 1)



/local/home/gmarsich/data2TB/Hypersim/evermotion_dataset/scenes/ai_007_008/images/scene_cam_00_final_hdf5/*.color.hdf5
['/local/home/gmarsich/data2TB/Hypersim/evermotion_dataset/scenes/ai_007_008/images/scene_cam_00_final_hdf5/frame.0000.color.hdf5']
Intrinsic Matrix for the scene:
[[1.43810735e+03 0.00000000e+00 5.12000000e+02]
 [0.00000000e+00 1.43810735e+03 3.83920000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [8]:
cloud = point_clouds[0]
print(cloud)

PointCloud with 786431 points.


In [9]:
# Assuming 'point_clouds' contains the list of point cloud objects
# Let's say the first element of the list is 'pcd'
pcd = point_clouds[0]

# Create a visualization window
vis = o3d.visualization.Visualizer()

# Add the point cloud to the visualization window
vis.create_window()
vis.add_geometry(pcd)

# Set the render options (optional)
render_options = vis.get_render_option()
render_options.point_size = 2  # Adjust the size of the points

# Render the visualization
vis.run()

# Close the visualization window
vis.destroy_window()


In [10]:


def merge_point_clouds(point_clouds):
    """
    Merge multiple point clouds into a single point cloud.
    
    Parameters:
        point_clouds (list): List of Open3D PointCloud objects.
    
    Returns:
        combined_pcd (open3d.geometry.PointCloud): Combined point cloud.
    """
    
    combined_pcd = o3d.geometry.PointCloud()
    for pcd in point_clouds:
        combined_pcd += pcd
    
    return combined_pcd

In [11]:
# Main function to execute the entire pipeline
def main():
    path_metadata = '/local/home/gmarsich/Desktop/Thesis/0Code_playground/pointClouds/pointClouds_ChatGPT/metadata_camera_parameters.csv'

    base_path = '/data2TB/Hypersim/evermotion_dataset/scenes'
    scene = 'ai_001_001'  # name of the scene, with format ai_VVV_NNN
    cam_xx = 'cam_00'

    print("TODO Here!")

    # Get list of image and depth HDF5 files
    image_files = sorted(glob.glob(os.path.join(base_path, scene, 'images', 'scene_' + cam_xx + '_final_hdf5', '*.color.hdf5')))
    depth_files = sorted(glob.glob(os.path.join(base_path, scene, 'images', 'scene_' + cam_xx + '_geometry_hdf5', '*.depth_meters.hdf5')))

    # Ensure the number of image and depth files match
    if len(image_files) != len(depth_files):
        raise ValueError("The number of image files and depth files do not match.")

    # Get the intrinsic matrix for the scene
    intrinsics = get_intrinsics_with_tilt_shift(path_metadata, scene)
    print("Intrinsic Matrix for the scene:")
    print(intrinsics)

    # Load images and depth data
    images, depths = load_hdf5_data(image_files, depth_files)

    # Generate point clouds for each view
    point_clouds = generate_point_cloud(images, depths, intrinsics)

    # Merge point clouds into a single point cloud
    combined_pcd = merge_point_clouds(point_clouds)

    # Save the point cloud to a file
    o3d.io.write_point_cloud("combined_point_cloud.pcd", combined_pcd)

    # Visualize the point cloud
    o3d.visualization.draw_geometries([combined_pcd])

if __name__ == "__main__":
    main()


TODO Here!
Intrinsic Matrix for the scene:
[[nan  0. nan]
 [ 0. nan nan]
 [ 0.  0.  1.]]


TypeError: generate_point_cloud() missing 1 required positional argument: 'extrinsic_matrices'